In [1]:
import sys
sys.path.append("..")

In [2]:
from solvers.MMR.PQK_solver import *
from circuits.circuits import *
from utils.rbf_kernel_tools import *
from squlearn.encoding_circuit import *


In [3]:
from squlearn.observables import SinglePauli
import numpy as np


def PQK_observable(num_qubits, measurement = "XYZ"):
    """"
    Returns the observable for the PQK solver

    Args:

    num_qubits (int): number of qubits in the system
    measurement (str): measurement operator to be applied to the qubits (default: "XYZ")

    Returns:
    _measurement (list): list of SinglePauli objects representing the measurement operator (shape: num_qubits*len(measurement))

    """
    if isinstance(measurement, str):
                _measurement = []
                for m_str in measurement:
                    if m_str not in ("X", "Y", "Z"):
                        raise ValueError("Unknown measurement operator: {}".format(m_str))
                    for i in range(num_qubits):
                        _measurement.append(SinglePauli(num_qubits, i, op_str=m_str))
    return _measurement

In [4]:
num_qubits = 2
num_features = 1
num_samples = 10
num_layers = 3

sigma = 1
measurement = "XYZ"
_measurement = PQK_observable(num_qubits, measurement)
print(len(_measurement), num_qubits*len(measurement))

6 6


In [5]:
#a = LowLevelQNN(Separable_rx(num_qubits=4, num_layers=1, num_features=2), _measurement, Executor("pennylane"))
#import RBF from sklearn
from sklearn.gaussian_process.kernels import RBF

a = LowLevelQNN(HubregtsenEncodingCircuit(num_qubits=num_qubits, num_layers=num_layers, num_features=num_features), _measurement, Executor("pennylane"))
parameter_array = np.random.rand(a.num_parameters)
print("num_samples", num_samples, "num_features", num_features, "num_parameters", a.num_parameters )
input = np.random.rand(num_samples, num_features)
X = a.evaluate(input, parameter_array, [], "f")["f"] # Shape (num_samples, num_qubits*len(measurement))

num_samples 10 num_features 1 num_parameters 6


In [6]:
O = a.evaluate(input, parameter_array, [], "f")["f"]
dOdx = a.evaluate(input, parameter_array, [], "dfdx")["dfdx"]
dOdxdx = a.evaluate(input, parameter_array, [], "dfdxdx")["dfdxdx"]

In [7]:
dOdx.shape

(10, 6, 1)

In [8]:
dOdxdx.shape

(10, 6, 1, 1)

In [9]:
X.shape

(10, 6)

In [10]:
from utils.rbf_kernel_tools import analytical_derivative_rbf_kernel, analytical_derivative_dot_kernel_2, matrix_rbf_dxdy_slow

from utils.rbf_kernel_tools import matrix_rbf, matrix_rbf_dx_slow, matrix_rbf_dxdx_slow

$K(x,y)$

In [11]:
K = RBF(1)(O, O)

Derivative of a kernel 

$\frac{d}{dx} k(x,y) = $

In [12]:
dOdxdx.shape


(10, 6, 1, 1)

In [13]:
K_rbf_dx =  matrix_rbf_dx_slow(X, X, sigma=sigma) #(n,n, d)
Kdx = np.einsum("ijk->ij",(K_rbf_dx*O))

$ \frac{d^{2} K}{dx^2}$

In [14]:
K_rbf_dxdy = matrix_rbf_dxdy_slow(X, X, sigma=sigma)

In [15]:
K_rbf_dxdx =  matrix_rbf_dxdx_slow(X, X, sigma=sigma)
K_rbf_dxdy = matrix_rbf_dxdy_slow(X, X, sigma=sigma)

Kdxdx = np.zeros((num_samples, num_qubits*len(measurement), num_features, num_features))
Kdxdx_first_term = np.einsum("ijk->ij",(K_rbf_dxdx*dOdx[:,:,0]))
Kdxdx_second_term = np.einsum("ijk->ij",(K_rbf_dx*dOdxdx[:,:,0,0]))
Kdxdx = Kdxdx_first_term + Kdxdx_second_term


In [16]:
Kdxdx

array([[ 2.25564227, -1.69887568,  0.92826903, -3.05591782, -1.96951338,
        -0.56549373, -0.56052573, -2.19758945, -3.10033274, -1.21426431],
       [-0.29303529,  0.68802725,  1.02073929, -3.83734488, -3.40062429,
         1.16159956,  1.16246774, -4.09187004, -3.63118708,  0.96402053],
       [ 1.68963984, -0.3566801 ,  1.76136921, -3.64583115, -2.50246369,
         0.78871534,  0.7929352 , -2.93785387, -3.60556357,  0.18038893],
       [-1.90605468, -1.04785096, -1.54441435, -2.40077736, -5.06852381,
        -1.24317515, -1.24411024, -5.36265713, -2.17305741, -1.125929  ],
       [-1.06931898, -2.02086062, -1.35456396, -3.71037714, -4.93409816,
        -1.67856056, -1.67731166, -4.63694376, -3.63571464, -1.85795485],
       [ 0.64766611,  0.38906283,  1.57319739, -3.8619705 , -2.97188188,
         1.22707752,  1.22958424, -3.56026371, -3.7332641 ,  0.81739002],
       [ 0.65149983,  0.38720951,  1.57481702, -3.8616717 , -2.97021715,
         1.22656814,  1.22908142, -3.55812286

In [17]:
Kdxdx_first_term.shape

(10, 10)

Testing implmentation

In [18]:
from circuits.circuits import Separable_rx

In [19]:
sigma = 1
gamma = 1/(2*sigma**2)

experiment = { "sigma": sigma}
experiment["circuit_information"] = {"encoding_circuit": Separable_rx, "num_qubits": 1, "num_layers":1}
pqk =PQK_solver(experiment["circuit_information"],
                                Executor("pennylane"), 
                                envelope={"function": matrix_rbf, 
                                            "derivative_function": matrix_rbf_dx_slow, 
                                            "second_derivative_function": matrix_rbf_dxdx_slow,
                                            "mixed_derivative_function": matrix_rbf_dxdy_slow,
                                            "sigma": experiment["sigma"]})

In [113]:
PQK_qnn = pqk.PQK_QNN()
obs_coef = []
X = [0.1, -0.41]
#X = [-0.41, 0.1]

K_f, K_dfdx, K_dfdxdx = pqk.get_PQK_kernel_derivatives(X, PQK_qnn, obs_coef, [1, 2])
#The problem is in analytical 2 and mixed term

[0.1, -0.41]
index_combinations_of_O [(0, 1), (0, 2), (1, 2)]
analytical 2 [[[-1.         -1.         -1.        ]
  [-0.88050868 -0.66175055 -0.87516768]]

 [[-0.88050868 -0.66175055 -0.87516768]
  [-1.         -1.         -1.        ]]]
dOdx square [[0.         0.99003329 0.00996671]
 [0.         0.8411106  0.1588894 ]]
first_term dOdx squared [[-1.         -0.69566027]
 [-0.66387762 -1.        ]]
second_term dOdxdx [[ 0.         -0.11204957]
 [-0.11204957  0.        ]]
mixed_term [[ 0.          0.00679085]
 [-0.0249918   0.        ]]


In [21]:
K_f

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [22]:
O = PQK_qnn.evaluate(X, [], [], "f")["f"]
dOdx = PQK_qnn.evaluate(X, [], [], "dfdx")["dfdx"]
dOdxdx = PQK_qnn.evaluate(X, [], [], "dfdxdx")["dfdxdx"]

In [23]:
matrix_rbf(O, O, sigma)[0, 1]

0.8805086806207679

In [24]:
O.shape

(2, 3)

In [25]:
O[0]

array([ 0.        , -0.09983342,  0.99500417])

In [26]:
dOdx.shape

(2, 3, 1)

In [27]:
K_dx_manual = np.zeros((len(X), len(X)))
for n in range(len(X)):
   for j in range(len(X)):
      r = 0
      for l in range(O.shape[1]):
         K_dx_manual[n,j] += matrix_rbf_dx_slow(O, O, sigma)[n,j, l] * dOdx[n, l, 0]

In [28]:
# Assuming dOdx has shape (N, L, 1) and the result of matrix_rbf_dx_slow has shape (N, N, L)
# Reshape dOdx to (N, L) to align the dimensions correctly for einsum
dOdx_reshaped = dOdx[:, :, 0]

# Calculate the result using einsum
K_dx_manual = np.einsum('njl, nl->nj', matrix_rbf_dx_slow(O, O, sigma), dOdx_reshaped)


In [29]:
import sympy as sp

x,y,gamma_sp = sp.symbols("x y gamma")

K = sp.exp(-gamma_sp*(2-2*sp.cos(x-y)))
Kx = sp.diff(K, x)
Kxx = sp.diff(Kx, x)

In [30]:
matrix_rbf_dxdy_slow(np.array([[0.5]]), np.array([[0.7]]), sigma=sigma)

array([[[[0.03920795]]]])

In [31]:
x1, y1 = sp.symbols('x_1 y_1')
x2, y2 = sp.symbols('x_2 y_2')

In [32]:
rbf_sp = sp.exp(-gamma_sp*((x1-y1)**2+(x2-y2)**2))

In [33]:
matrix_rbf_dx_slow(np.array([[0.5, 0.4]]), np.array([[0.7, 0.3]]), sigma=sigma)

array([[[ 0.19506198, -0.09753099]]])

In [34]:
print(rbf_sp.diff(x1).subs({x1: 0.5, y1: 0.7, x2: 0.4, y2: 0.3, gamma_sp: gamma}))
print(rbf_sp.diff(x2).subs({x1: 0.5, y1: 0.7, x2: 0.4, y2: 0.3, gamma_sp: gamma}))

0.195061982405666
-0.0975309912028333


In [35]:
print(rbf_sp.diff(x2).diff(x1).subs({x1: 0.5, y1: 0.7, x2: 0.4, y2: 0.3, gamma_sp: gamma}))

-0.0195061982405667


In [36]:
matrix_rbf_dxdy_slow(np.array([[0.5, 0.4]]), np.array([[0.7, 0.3]]), sigma=sigma)

array([[[[ 0.0390124, -0.0195062],
         [-0.0195062,  0.0097531]]]])

In [37]:
rbf_sp.diff(x).diff(y)

0

In [38]:
rbf_sp.diff(x).subs({x: 0.5, y: 0.7, gamma_sp: gamma})

0

In [39]:
x1, y1 = sp.symbols('x_1 y_1')
x2, y2 = sp.symbols('x_2 y_2')
O1 = sp.symbols('O_1', cls=sp.Function)
O2 = sp.symbols('O_2', cls=sp.Function)
O3 = sp.symbols('O_3', cls=sp.Function)


In [40]:
k = sp.symbols('k', cls=sp.Function)

In [41]:
O2_ = sp.sin
O3_ = sp.cos
rbf_sp = sp.exp(-gamma_sp*((x1-y1)**2+(x2-y2)**2))

def rbf_sp_fun(x1, x2, y1, y2):
    return sp.exp(-gamma_sp*((x1-y1)**2+(x2-y2)**2))

In [42]:
#k = rbf_sp_fun
pqk_fun =  k(O2(x), O3(x),O2(y), O3(y))
pqk_fun_dx = sp.diff(pqk_fun, x)

In [43]:
pqk_fun_dx.subs({O2:O2_, O3:O3_}).doit().simplify()

-sin(x)*Subs(Derivative(k(sin(x), _xi_0, sin(y), cos(y)), _xi_0), _xi_0, cos(x)) + cos(x)*Subs(Derivative(k(_xi_0, cos(x), sin(y), cos(y)), _xi_0), _xi_0, sin(x))

In [44]:
Kx

-2*gamma*exp(-gamma*(2 - 2*cos(x - y)))*sin(x - y)

In [45]:
((k(O2(x), O3(x),  O2(y), O3(y))).diff(x)).subs(k, rbf_sp)

Derivative(O_2(x), x)*Derivative(k(O_2(x), O_3(x), O_2(y), O_3(y)), O_2(x)) + Derivative(O_3(x), x)*Derivative(k(O_2(x), O_3(x), O_2(y), O_3(y)), O_3(x))

In [46]:
first_part = (k(O2(x), O3(x), O2(y), O3(y)).diff(x, 2)).simplify().args[:2]
second_part = (k(O2(x), O3(x), O2(y), O3(y)).diff(x, 2)).simplify().args[2:4]
mixed_part = (k(O2(x), O3(x), O2(y), O3(y)).diff(x, 2)).simplify().args[4]
everything = (k(O2(x), O3(x), O2(y), O3(y)).diff(x, 2)).simplify()

In [47]:
(k(O2(x), O3(x), O2(y), O3(y)).diff(x, 2)).simplify().args[0]

Derivative(O_2(x), x)**2*Derivative(k(O_2(x), O_3(x), O_2(y), O_3(y)), (O_2(x), 2))

In [48]:
def get_num_expr(expr, x, y, gamma):
    new_expr = expr.subs(k(O2(x), O3(x), O2(y), O3(y)), rbf_sp).subs({x1:O2(x), x2:O3(x), y1:O2(y), y2:O3(y)}).doit().subs({O2:O2_, O3:O3_}).doit().subs({gamma_sp: gamma}).simplify()
    return new_expr

sp_K_PQK_dxdx = everything.subs(k(O2(x), O3(x), O2(y), O3(y)), rbf_sp).subs({x1:O2(x), x2:O3(x), y1:O2(y), y2:O3(y)}).doit().subs({O2:O2_, O3:O3_}).doit().subs({gamma_sp: gamma}).simplify()
first_part_num = [get_num_expr(first_part[0], x, y, gamma), get_num_expr(first_part[1], x, y, gamma)]
second_part_num = [get_num_expr(second_part[0], x, y, gamma), get_num_expr(second_part[1], x, y, gamma)]
mixed_part_num = get_num_expr(mixed_part, x, y, gamma)

In [49]:
first_part_num[0]

0.367879441171442*((sin(x) - sin(y))**2 - 1)*exp(1.0*cos(x - y))*cos(x)**2

In [50]:
first_part[0].subs(k(O2(x), O3(x), O2(y), O3(y)), rbf_sp).subs({x1:O2(x), x2:O3(x), y1:O2(y), y2:O3(y)}).doit().subs({O2:O2_, O3:O3_}).doit()

2*gamma*(2*gamma*(sin(x) - sin(y))**2 - 1)*exp(-gamma*((sin(x) - sin(y))**2 + (cos(x) - cos(y))**2))*cos(x)**2

In [51]:
first_part[1].subs(k(O2(x), O3(x), O2(y), O3(y)), rbf_sp).subs({x1:O2(x), x2:O3(x), y1:O2(y), y2:O3(y)}).doit().subs({O2:O2_, O3:O3_}).doit()

2*gamma*(2*gamma*(cos(x) - cos(y))**2 - 1)*exp(-gamma*((sin(x) - sin(y))**2 + (cos(x) - cos(y))**2))*sin(x)**2

In [52]:
second_part[0].subs(k(O2(x), O3(x), O2(y), O3(y)), rbf_sp).subs({x1:O2(x), x2:O3(x), y1:O2(y), y2:O3(y)}).doit().subs({O2:O2_, O3:O3_}).doit()

gamma*(2*sin(x) - 2*sin(y))*exp(-gamma*((sin(x) - sin(y))**2 + (cos(x) - cos(y))**2))*sin(x)

In [53]:
second_part[1].subs(k(O2(x), O3(x), O2(y), O3(y)), rbf_sp).subs({x1:O2(x), x2:O3(x), y1:O2(y), y2:O3(y)}).doit().subs({O2:O2_, O3:O3_}).doit()

gamma*(2*cos(x) - 2*cos(y))*exp(-gamma*((sin(x) - sin(y))**2 + (cos(x) - cos(y))**2))*cos(x)

In [54]:
mixed_part.subs(k(O2(x), O3(x), O2(y), O3(y)), rbf_sp).subs({x1:O2(x), x2:O3(x), y1:O2(y), y2:O3(y)}).doit().subs({O2:O2_, O3:O3_}).doit()

-8*gamma**2*(sin(x) - sin(y))*(cos(x) - cos(y))*exp(-gamma*((sin(x) - sin(y))**2 + (cos(x) - cos(y))**2))*sin(x)*cos(x)

In [55]:
def K_separable_rx_PQK_(X,Y, gamma):
    gram_matrix = np.zeros((len(X), len(Y)))    
    for i in range(len(X)):
        for j in range(len(Y)):
            gram_matrix[i,j] = np.exp(-gamma*(2-2*np.cos(X[i]-Y[j])))
    return gram_matrix

def K_separable_rx_PQK_dx(X,Y, gamma):
    #-2*gamma*exp(-gamma*(2 - 2*cos(x - y)))*sin(x - y)

    gram_matrix = np.zeros((len(X), len(Y)))    
    for i in range(len(X)):
        for j in range(len(Y)):
            gram_matrix[i,j] = -2*gamma*np.exp(-gamma*(2-2*np.cos(X[i]-Y[j])))*np.sin(X[i]-Y[j])
    return gram_matrix 

def K_separable_rx_PQK_dxdx(X,Y, gamma):
    #4*gamma**2*exp(-gamma*(2 - 2*cos(x - y)))*sin(x - y)**2 - 2*gamma*exp(-gamma*(2 - 2*cos(x - y)))*cos(x - y)
    gram_matrix = np.zeros((len(X), len(Y)))    
    for i in range(len(X)):
        for j in range(len(Y)):
            gram_matrix[i,j] = 4*gamma**2*np.exp(-gamma*(2-2*np.cos(X[i]-Y[j])))*np.sin(X[i]-Y[j])**2 - 2*gamma*np.exp(-gamma*(2-2*np.cos(X[i]-Y[j])))*np.cos(X[i]-Y[j])
    return gram_matrix 

In [56]:
from squlearn.kernel.matrix import ProjectedQuantumKernel

In [57]:
pqk_squlearn = ProjectedQuantumKernel(Separable_rx(num_qubits=1, num_layers=1), executor=Executor("pennylane"), gamma=gamma)

In [58]:
pqk_squlearn.evaluate(X, X)

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [59]:
K_separable_rx_PQK_(X,X, gamma)

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [60]:
K_f

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [61]:
K_separable_rx_PQK_dx(X,X,gamma)

array([[-0.       , -0.4298443],
       [ 0.4298443, -0.       ]])

In [62]:
K_dfdx

array([[ 0.       , -0.4298443],
       [ 0.4298443,  0.       ]])

In [63]:
K_separable_rx_PQK_dxdx(X,X,gamma)

array([[-1.        , -0.55861891],
       [-0.55861891, -1.        ]])

In [64]:
K_dfdxdx

array([[-1.        , -0.80091898],
       [-0.80091898, -1.        ]])

In [65]:
first_part_num[0].subs({x: X[0], y: X[1]}) + first_part_num[1].subs({x: X[0], y: X[1]}) + second_part_num[0].subs({x: X[0], y: X[1]}) + second_part_num[1].subs({x: X[0], y: X[1]}) + mixed_part_num.subs({x: X[0], y: X[1]})

-0.558618906245010

In [66]:
print("first part", first_part_num[0].subs({x: X[0], y: X[1]}) + first_part_num[1].subs({x: X[0], y: X[1]}))
print("second part", second_part_num[0].subs({x: X[0], y: X[1]}) + second_part_num[1].subs({x: X[0], y: X[1]}))
print("mixed part", mixed_part_num.subs({x: X[0], y: X[1]}))
print("total", first_part_num[0].subs({x: X[0], y: X[1]}) + first_part_num[1].subs({x: X[0], y: X[1]}) + second_part_num[0].subs({x: X[0], y: X[1]}) + second_part_num[1].subs({x: X[0], y: X[1]}) + mixed_part_num.subs({x: X[0], y: X[1]}))

first part -0.663877618895062
second part 0.112049565674586
mixed part -0.00679085302453363
total -0.558618906245010


In [67]:
sp_K_PQK_dxdx.subs({x: X[0], y: X[1]})

-0.558618906245010

In [79]:
everything

Derivative(O_2(x), x)**2*Derivative(k(O_2(x), O_3(x), O_2(y), O_3(y)), (O_2(x), 2)) + 2*Derivative(O_2(x), x)*Derivative(O_3(x), x)*Derivative(k(O_2(x), O_3(x), O_2(y), O_3(y)), O_2(x), O_3(x)) + Derivative(O_2(x), (x, 2))*Derivative(k(O_2(x), O_3(x), O_2(y), O_3(y)), O_2(x)) + Derivative(O_3(x), x)**2*Derivative(k(O_2(x), O_3(x), O_2(y), O_3(y)), (O_3(x), 2)) + Derivative(O_3(x), (x, 2))*Derivative(k(O_2(x), O_3(x), O_2(y), O_3(y)), O_3(x))

In [84]:
get_num_expr(first_part[0].args[0], x, y, gamma).subs({x: X[0], y: X[1]}), get_num_expr(first_part[1].args[0], x, y, gamma).subs({x: X[1], y: X[1]}), 

(0.990033288920621, 0.158889396356193)

In [78]:
first_part[0].args[0]

Derivative(O_2(x), x)**2

In [97]:
dummy_matrix = np.zeros((2,2,2))
for l in range(2):
    for j in range(2):
        for i in range(2):
            dummy_matrix[i,j,l] = get_num_expr(first_part[l].args[1], x, y, gamma).subs({x: X[i], y: X[j]})


In [98]:
dummy_matrix

array([[[-1.        , -1.        ],
        [-0.66175055, -0.87516768]],

       [[-0.66175055, -0.87516768],
        [-1.        , -1.        ]]])

In [72]:
first_part[0].args[1]

Derivative(k(O_2(x), O_3(x), O_2(y), O_3(y)), (O_2(x), 2))

In [73]:
index_combinations_of_O [(0, 1), (0, 2), (1, 2)]
analytical 2 [[[-1.         -1.         -1.        ]
  [-0.88050868 -0.66175055 -0.87516768]]

 [[-0.88050868 -0.66175055 -0.87516768]
  [-1.         -1.         -1.        ]]]
dOdx square [[0.         0.99003329 0.00996671]
 [0.         0.8411106  0.1588894 ]]
first_term dOdx squared [[-1.         -0.69566027]
 [-0.66387762 -1.        ]]
second_term dOdxdx [[ 0.         -0.11204957]
 [-0.11204957  0.        ]]
mixed_term [[ 0.          0.00679085]
 [-0.0249918   0.        ]]

SyntaxError: invalid syntax (1424646232.py, line 2)